# Project data analisys m1

##### Problmeas encontrados y cómo se resolvieron
### Readme con fotos e instrucciones
Importar librerías
### Adquisition
Importar datos, sql y api
### Wrangling
Limpieza de datos. Tengo dos nulos: 192(191) y 98(99)
### Analisys
Crear funciones siempre mejor

Clacular distancias y limpiar

Asignar la station y address a places

### Reporting
Two options: all place or one

To csv

Additional documents: pdf, power points,...

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import re
import duckdb
from shapely.geometry import Point
import geopandas as gpd
from modules import geo_calculations
pd.options.display.max_columns = None
pd.options.display.max_rows = None

| Place of interest | Type of place (*) | Place address | BiciMAD station | Station location |

---
### Gardens

In [2]:
# endpoint api Madrid gardens
response = requests.get('https://datos.madrid.es/egob/catalogo/200761-0-parques-jardines.json')
results_api = response.json()

In [3]:
# Selecciono la lista que quiero
list_api_garden = results_api['@graph']

In [4]:
# Transformo a data frame
df_garden = pd.DataFrame(list_api_garden)

In [5]:
# Separo langitud y latitud de la columna location
df_garden_location = df_garden["location"].apply(pd.Series)

In [6]:
# Separa los elementos de address
df_garden_address = df_garden["address"].apply(pd.Series)

In [7]:
# Elimino lo innecesario para hacerlo más legible de @type
# La barra se podría cambiar con un str.replace por una coma
df_garden_type = df_garden["@type"].str.replace('https://datos.madrid.es/egob/kos/entidadesYorganismos/', '')

In [8]:
# En el data frame de garden, me quedo solo con title, para no repetir @type y evitar posibles problemas en el futuro
both_df_garden = pd.concat([df_garden[['title']], df_garden_location, df_garden_address, df_garden_type], axis = 1)

In [9]:
# Elimino columnas o selecciono columnas que quiero: location, type, address and title of gardens, o no
cleaning_df_garden = both_df_garden[['title', '@type', 'locality', 'postal-code', 'street-address', 'latitude', 'longitude']]

---
### Bicimad

In [10]:
# endpoint duck

In [11]:
db_bicimad = duckdb.connect('bicimad.db')

In [12]:
# Data Frame data base statiosn

In [13]:
df_bicimad = db_bicimad.sql('select * from bicimad_stations').df()

In [14]:
# Separo geometry coordinates

In [15]:
df_bicimad[['longitude', 'latitude']] = df_bicimad["geometry.coordinates"].apply(lambda x: pd.Series(str(x).replace('[','').replace(']','').split(",")))

In [16]:
df_bicimad = df_bicimad[['name', 'address', 'latitude', 'longitude',]]

---

## Muestro los dos resultados de data wrangling

In [17]:
df_bicimad

,name,address,latitude,longitude
0,1a - Puerta del Sol A,Puerta del Sol nº 1,40.4172137,-3.7018341
1,1b - Puerta del Sol B,Puerta del Sol nº 1,40.41731271011562,-3.701602938060457
2,2 - Miguel Moya,Calle Miguel Moya nº 1,40.4205886,-3.7058415
3,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.4302937,-3.7069171
4,4 - Malasaña,Calle Manuela Malasaña nº 5,40.4285524,-3.7025875
5,5 - Fuencarral,Calle Fuencarral nº 108,40.42852,-3.70205
6,6 - Colegio Arquitectos,Calle Hortaleza nº 63,40.424148,-3.698447
7,7 - Hortaleza,Calle Hortaleza nº 75,40.4251906,-3.6977715
8,8 - Alonso Martínez,Plaza de Alonso Martínez nº 5,40.4278682,-3.6954403
9,9 - Plaza de San Miguel,Plaza de San Miguel nº 9,40.4156057,-3.7095084


In [18]:
cleaning_df_garden

,title,@type,locality,postal-code,street-address,latitude,longitude
0,Jardines Gregorio Ordóñez,ParquesJardines,MADRID,28006,CALLE PRINCIPE DE VERGARA 68,40.433462,-3.678595
1,Jardines San Francisco el Grande,ParquesJardines/ColeccionesBotanicas,MADRID,28005,AVENIDA GRAN VIA DE SAN FRANCISCO 29,40.407143,-3.712167
2,Jardines de Andrés Saborit,ParquesJardines,MADRID,28003,CALLE RAIMUNDO FERNANDEZ VILLAVERDE 32,40.446133,-3.698970
3,Jardines de El Buen Retiro,ParquesJardines/ParquesHistoricos,MADRID,28001,PLAZA INDEPENDENCIA 7,40.419641,-3.687934
4,Jardines de Gloria Fuertes,NaN,MADRID,28016,CALLE MADRE DE DIOS 2,40.461518,-3.675118
5,Jardines de José Luis Jiménez Manso,ParquesJardines,MADRID,28030,CALLE CAMINO DE LOS VINATEROS 1,40.410945,-3.660502
6,Jardines de Perón,ParquesJardines,MADRID,28020,AVENIDA GENERAL PERON 21,40.452576,-3.695782
7,Jardines de Sabatini,ParquesJardines,MADRID,28013,CALLE BAILEN 2,40.419911,-3.713185
8,Jardines de San Fernando,ParquesJardines,MADRID,28036,AVENIDA ALBERTO ALCOCER 9,40.459788,-3.687352
9,Jardines de la Avenida de Brasil,ParquesJardines,MADRID,28020,AVENIDA BRASIL 31,40.458707,-3.693463


## Data Analisys

In [19]:
# Incluir de alguna forma la comparacion con un loop o comprehension

In [20]:
# Incluyo las funciones para pruebas

In [21]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [22]:
# Mido las distancias

In [23]:
%%time
distance=[]
for index, row in cleaning_df_garden.iterrows():
    for index, j in df_bicimad.iterrows():
        distance.append(distance_meters(float(row['latitude']), float(row['longitude']), float(j['latitude']), float(j['longitude'])))

CPU times: total: 10min 43s
Wall time: 11min 55s


In [24]:
%%time
list_df_coordinates=[]
count=0
list_list = []
for i in distance:
    if count != 264:
        count = count + 1
        list_list.append(i)
    if count == 264:
        count = 0
        list_df_coordinates.append(list_list)
        list_list = []

CPU times: total: 0 ns
Wall time: 31.1 ms


In [25]:
df_coordinates = pd.DataFrame(list_df_coordinates)

In [26]:
%%time
def remove_non_numeric(s):
    return re.sub('[^0-9\.]+', '', s)

# convert all values in the dataframe to strings
df_coordinates = df_coordinates.astype(str)

# apply the function to all elements in the dataframe using a lambda function
df_coordinates = df_coordinates.applymap(lambda x: remove_non_numeric(x))

# convert all values in the dataframe back to float data type
df_coordinates = df_coordinates.astype(float)

CPU times: total: 2.73 s
Wall time: 15.1 s


In [27]:
# mínimo de las filas
min_rows = df_coordinates.min(axis=1)

In [28]:
dict_min_distances = min_rows.to_dict()

In [29]:
dict_all_distances = df_coordinates.to_dict()

In [30]:
dict_min_station = {}
for keys1, values1 in dict_all_distances.items():
    for key_a, value_a in values1.items():
        for key_min, value_min in dict_min_distances.items():
            if value_a == value_min:
                dict_min_station[key_min] = (value_min, keys1)
                
len(dict_min_station)

203

In [31]:
dict_min_station = dict(sorted(dict_min_station.items())) # place: distance,station

In [32]:
df_min_station=pd.DataFrame.from_dict(dict_min_station, orient='index')

In [33]:
df_min_station.rename(columns={1: "1a"}, inplace=True)

In [34]:
df_bicimad['index'] = df_bicimad.index


In [35]:
df_bicimad.rename(columns={"index": "1a"}, inplace=True)


In [36]:
list_min_station_names = {}
count = 0
for index, row1 in df_min_station.iterrows():
    for index, row2 in df_bicimad.iterrows():
        if row1['1a'] == row2['1a']:
            list_min_station_names[count]= (row2['name'], row2['address'])
            count = count +1


In [37]:
df_min_station_names = pd.DataFrame(list_min_station_names)

In [38]:
df_min_station_names = df_min_station_names.transpose()

In [39]:
df_min_station_names.rename(columns={0: "name1", 1:'address1'}, inplace=True)

In [40]:
df_min_station_names1 = df_min_station_names['name1'].apply(lambda x: pd.Series(str(x).split('-')))

In [41]:
df_min_station_names1.rename(columns={0: "numberof", 1:'name2', 2:'null'}, inplace=True)

In [42]:
df_final = pd.concat([cleaning_df_garden, df_min_station_names, df_min_station_names1], axis=1)

In [43]:
df_final = df_final[['title', '@type', 'street-address', 'name2', 'address1']]

In [44]:
df_final = df_final.rename(columns={'title': 'Place of interest', '@type': 'Type of place', 'street-address': 'Place address', 'name2': 'BiciMAD station', 'address1': 'Station location'})

In [46]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Place of interest  203 non-null    object
 1   Type of place      183 non-null    object
 2   Place address      203 non-null    object
 3   BiciMAD station    203 non-null    object
 4   Station location   203 non-null    object
dtypes: object(5)
memory usage: 8.1+ KB


In [ ]:
# Cambiar los null por Jardínes